In [1]:
# Import libraries

import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn.svm as svm
import sklearn.pipeline as spl
import sklearn.kernel_ridge as skr
import sklearn.model_selection as sms
import sklearn.linear_model as slm
import sklearn.preprocessing as skp
import sklearn.neighbors as snn
import sklearn.metrics as sme
import sklearn.decomposition as sdc
import sklearn.cross_decomposition as skd
import sklearn.feature_selection as skf
import sklearn.ensemble as ske
import sklearn.utils as sku
from sklearn.utils import resample
from sklearnex import patch_sklearn, config_context
from sklearn.cluster import DBSCAN
import numpy as np
import scipy.stats as stats
from IPython.display import HTML
import util
from sklearn import metrics
import nibabel as nib
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
HTML('''
<style>
.jupyter-matplotlib {
    background-color: #000;
}

.widget-label, .jupyter-matplotlib-header{
    color: #fff;
}

.jupyter-button {
    background-color: #333;
    color: #fff;
}
</style>
''')

In [3]:
# Maybe y should also be scaled? 
#     Performed poorly using Standard and MinMax scalers. Trying with LOOCV to see if predictions stabilize.
#     Does not appear to stabilize predictions with LOOCV (using StandardScaler())
# Perhaps a transform would be more effective, or scaling implemented with consistent cross-validation
# Different scaling methods? 
#     This seems most important for noise-sensitive models like LARS. All other use StandardScaler()
# Transformers?
# Model-specific scaling methods?
#     Yes, see above
# Common cross-validation function?
#     Use built-in functions wherever possible and `utils.gridsearch_pickparams()` elsewhere
# Quantile loss
# RANSAC
# Data augmentation? (Mixup)
# Data generation? (SMOGN)
# CHECK PHI AND X DIRECTORIES, WHICH ONE IS RIGHT?

In [4]:
reload = 1
qsms = util.full_path('/home/ali/RadDBS-QSM/data/nii/qsm')
qsms_subs = qsms[-9]
segs = util.full_path('/home/ali/RadDBS-QSM/data/nii/seg')
chi = []
im_subs = []
c = []
roi_s = np.asarray((1.0,2.0,3.0,4.0))
for j in np.arange(len(qsms)):
    data = nib.load(qsms[j])
    qsm_subs = qsms[j][-9:-7]
    #try:
    mask = nib.load('/home/ali/RadDBS-QSM/data/nii/seg/labels_2iMag'+qsm_subs+'.nii.gz').get_fdata()
    print(np.unique(mask))
    mask[mask > 4] = 0
    mask[mask < 1] = 0
    print(mask.shape)
    print(np.unique(mask))
    mask_k = []
    k_all = []
    for k in np.arange(mask.shape[2]):
            if np.sum(mask[:,:,k]) > 0:
                print('Appending slice',str(k),'with ROIs',str(np.unique(mask)))
                mask_k.append(np.sum(mask[:,:,k]))
                k_all.append(k)
        
    c.append(k_all[np.argmax(mask_k)])
    print('Maximum volume found at slice',str(k_all[np.argmax(mask_k)]))
#except:
#    print('Missing mask at',str(qsm_subs))



[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
(512, 512, 352)
[0. 1. 2. 3. 4.]
Appending slice 0 with ROIs [0. 1. 2. 3. 4.]
Appending slice 1 with ROIs [0. 1. 2. 3. 4.]
Appending slice 2 with ROIs [0. 1. 2. 3. 4.]
Appending slice 3 with ROIs [0. 1. 2. 3. 4.]
Appending slice 4 with ROIs [0. 1. 2. 3. 4.]
Appending slice 5 with ROIs [0. 1. 2. 3. 4.]
Appending slice 6 with ROIs [0. 1. 2. 3. 4.]
Appending slice 7 with ROIs [0. 1. 2. 3. 4.]
Appending slice 8 with ROIs [0. 1. 2. 3. 4.]
Appending slice 9 with ROIs [0. 1. 2. 3. 4.]
Appending slice 10 with ROIs [0. 1. 2. 3. 4.]
Appending slice 11 with ROIs [0. 1. 2. 3. 4.]
Appending slice 12 with ROIs [0. 1. 2. 3. 4.]
Appending slice 13 with ROIs [0. 1. 2. 3. 4.]
Appending slice 14 with ROIs [0. 1. 2. 3. 4.]
Appending slice 15 with ROIs [0. 1. 2. 3. 4.]
Appending slice 16 with ROIs [0. 1. 2. 3. 4.]
Appending slice 17 with ROIs [0. 1. 2. 3. 4.]
Appending slice 18 with ROIs [0. 1. 2. 3. 4.]
Appending slice 19 with ROIs [0. 

KeyboardInterrupt: 

In [ ]:
# Get case IDs
case_list = open('/home/ali/RadDBS-QSM/data/docs/cases_90','r')
lines = case_list.read()
lists = np.loadtxt(case_list.name,comments="#", delimiter=",",unpack=False,dtype=str)
case_id = []
for lines in lists:     
    case_id.append(lines[-9:-7])

# Load scores
file_dir = '/home/ali/RadDBS-QSM/data/docs/QSM anonymus- 6.22.2023-1528.csv'
motor_df = util.filter_scores(file_dir,'pre-dbs updrs','stim','CORNELL ID')
# Find cases with all required scores
subs_init,pre_imp_init,post_imp_init,pre_updrs_off_init = util.get_full_cases(motor_df,
                                                          'CORNELL ID',
                                                          'OFF (pre-dbs updrs)',
                                                          'ON (pre-dbs updrs)',
                                                          'OFF meds ON stim 6mo')
# Load extracted features
npy_dir = '/home/ali/RadDBS-QSM/data/npy/slices/'
phi_dir = '/home/ali/RadDBS-QSM/data/phi/slices/'
roi_path = '/data/Ali/atlas/mcgill_pd_atlas/PD25-subcortical-labels.csv'
n_rois = 6
Phi_all, X_all, R_all, K_all, ID_all = util.load_featstruct(phi_dir,npy_dir+'X/',npy_dir+'R/',npy_dir+'K/',n_rois,939,True)
ids = np.asarray(ID_all).astype(int)

# Find overlap between scored subjects and feature extraction cases
c_cases = np.intersect1d(np.asarray(case_id).astype(int),np.asarray(subs_init).astype(int))
# Complete case indices with respect to feature matrix
c_cases_idx = np.in1d(ids,c_cases)
X_all_c = X_all[c_cases_idx,:,:]
# K_all_c = K_all[c_cases_idx,:,:]
# R_all_c = R_all[c_cases_idx,:,:]
# Re-index the scored subjects with respect to complete cases
s_cases_idx = np.in1d(subs_init,ids[c_cases_idx])
subs_init = subs_init[s_cases_idx]
pre_imp_init = pre_imp_init[s_cases_idx]
post_imp_init = post_imp_init[s_cases_idx]
pre_updrs_off_init = pre_updrs_off_init[s_cases_idx]
per_change_init = post_imp_init
subs = np.asarray(ID_all,dtype=float)[np.in1d(np.asarray(ID_all,dtype=float),subs_init)]

pre_imp = np.zeros((1,len(subs))).T
post_imp = np.zeros((1,len(subs))).T
pre_updrs_off = np.zeros((1,len(subs))).T
per_change = np.zeros((1,len(subs))).T
for j in np.arange(len(subs)):
    pre_imp[j] = pre_imp_init[subs_init == subs[j]]
    post_imp[j] = post_imp_init[subs_init == subs[j]]
    pre_updrs_off[j] = pre_updrs_off_init[subs_init == subs[j]]
    per_change[j] = per_change_init[subs_init == subs[j]]

Appended 1000 slices
Appended 2000 slices
Appended 3000 slices
Appended 4000 slices
Appended 5000 slices
Appended 6000 slices
Allocated arrays


TypeError: list indices must be integers or slices, not tuple

In [ ]:
np.sum(~np.isnan(X_all))/np.prod(X_all.shape)

In [ ]:
#alphas = np.logspace(-9,-2,10)
scoring = 'neg_mean_squared_error'
results_lr = np.zeros_like(per_change_init)

In [ ]:
for j in np.arange(len(subs_init)):
    test_id = subs_init[j]
    test_index = np.argmax(subs == test_id)
    train_index = np.argmax(subs != test_id)
    print('With patient ID',str(np.unique(subs[test_index])),'pre-improvement is',str(np.unique(pre_updrs_off[test_index])))
    # print(len(test_index)) 
    # print(np.sum(test_index))
    # print(len(train_index))
    # print(np.sum(train_index))
    # print(np.unique(subs[test_index]))
    # print(test_id)
    X_train = X_all_c[train_index,:,:]
    X_test = X_all_c[test_index,:,:]
    y_train = per_change[train_index]
    y_test = per_change[test_index]
    X_train, y_train = sku.shuffle(X_train, y_train, random_state=1)

    # Cross validation
    X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                X_train,train_index,X_test,test_index,pre_updrs_off,True,False,True)
    # print(np.unique(pre_updrs_off[test_index]))
    cvn = 10
    with np.errstate(divide='ignore', invalid='ignore'):
        sel = skf.SelectKBest(skf.r_regression,k=400)
        X0_ss = sel.fit_transform(X0_ss0,y_train.ravel())
        X_test_ss = sel.transform(X_test_ss0)
    # Linear regression
    lr = slm.LassoCV(max_iter=1e7,verbose=True,n_jobs=-1)
    est_lr = lr.fit(X0_ss,y_train.ravel())
    print(est_lr.score(X0_ss,y_train))
    #est_lr.predict(X_test_ss)
    print('Linear regression predicts',str(np.mean(est_lr.predict(X_test_ss))),'for case with',str(per_change_init[j]))

    # # LASSO
    # lasso = slm.Lasso(alpha=1e-2,max_iter=1e5)
        
    # #verbose=False,
    # # random_state=1,
    # # max_iter=10000,
    # # tol=1e-3,
    # # n_jobs=-1)
    # est_ls = lasso.fit(X0_ss,y_train)
    # results_ls[j] = est_ls.predict(X_test_ss)
    # print('Lasso predicts',str(results_ls[j]),'for case with',str(per_change[j]))




In [ ]:
# _, _, coefs = slm.lars_path(X0_ss0, y_train, method="lasso",  verbose=True, max_iter = 104)

# xx = np.sum(np.abs(coefs.T), axis=1)
# xx /= xx[-1]

# plt.plot(xx, coefs.T)
# ymin, ymax = plt.ylim()
# plt.vlines(xx, ymin, ymax, linestyle="dashed")
# plt.xlabel("|coef| / max|coef|")
# plt.ylabel("Coefficients")
# plt.title("LASSO Path")
# plt.axis("tight")
# plt.show()


In [ ]:
len(per_change)

In [ ]:
util.eval_prediction(np.vstack((pre_imp_init,
                               results_lr)),
                               per_change_init,
                               ['LCT',
                                'Regression',
                                'Ridge',
                                ],(30,5))
plt.ylim([0,2])
plt.xlim([0,2])